# Buraitoraito

Buraitoraito ("Bright Light") is played on a rectangular or square grid. It contains black cells with numbers. The number in the cell represents how many stars can be seen from this cell. A star is visible from the black cell, if it is in the same row or column as this cell, but not behind other black cells.

![](https://www.janko.at/Raetsel/Buraitoraito/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Buraitoraito/Regeln-02.gif)




In [1]:
from collections import deque
from itertools import chain
# ===== vsualzie ===== 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib.pyplot import MultipleLocator


In [3]:
def readGrid(path):
    with open(f"../assets/data/Buraitoraito/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("problems/1_8x8")
    
    for g in grid:
        print(g)

['2', '-', '-', '5', '-', '-', '1', '-']
['-', '-', '4', '-', '-', '-', '-', '-']
['-', '4', '-', '-', '-', '-', '4', '-']
['1', '-', '-', '-', '-', '1', '-', '4']
['-', '-', '5', '-', '6', '-', '-', '-']
['3', '-', '-', '7', '-', '3', '-', '-']
['-', '-', '-', '-', '-', '-', '5', '-']
['-', '-', '2', '-', '-', '2', '-', '2']


In [9]:
from ortools.sat.python import cp_model as cp
def buraitoraito_solver(grid, X, Y):
    model = cp.CpModel()
    x = {}
    for i in range(X):
        for j in range(Y):
            if grid[i][j] in ["-"]:
                x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            else:
                x[i, j] = 0
    for i in range(X):
        for j in range(Y):
            if grid[i][j] not in ["-"]:
                num_cnt = int(grid[i][j])
                cand_pos = []
                tmp_i, tmp_j = i + 1, j
                while tmp_i < X:
                    if grid[tmp_i][tmp_j] not in ["-"]:
                        break
                    cand_pos.append((tmp_i, tmp_j))
                    tmp_i += 1
                tmp_i, tmp_j = i, j + 1
                while tmp_j < Y:
                    if grid[tmp_i][tmp_j] not in ["-"]:
                        break
                    cand_pos.append((tmp_i, tmp_j))
                    tmp_j += 1
                tmp_i, tmp_j = i - 1, j
                while tmp_i >= 0:
                    if grid[tmp_i][tmp_j] not in ["-"]:
                        break
                    cand_pos.append((tmp_i, tmp_j))
                    tmp_i -= 1

                tmp_i, tmp_j = i, j - 1
                while tmp_j >= 0:
                    if grid[tmp_i][tmp_j] not in ["-"]:
                        break
                    cand_pos.append((tmp_i, tmp_j))
                    tmp_j -= 1
                model.Add(sum([x[p] for p in cand_pos]) == num_cnt)

    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(X):
            for j in range(Y):
                if grid[i][j] not in "0123456789":
                    if solver.Value(x[i, j]) > 1e-1:
                        print("X", end=" ")
                    else:
                        print("O", end=" ")
                else:
                    print(grid[i][j], end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
    else:
        print("Something Wrong!")

if __name__ == "__main__":
    m, n, grid = readGrid("problems/1_8x8")
    buraitoraito_solver(grid, m, n)

2 X X 5 O O 1 X 
O O 4 X X O O O 
O 4 X X O O 4 X 
1 O O O O 1 O 4 
X X 5 X 6 X X X 
3 O X 7 X 3 O O 
X O O X O X 5 X 
O O 2 X O 2 O 2 

NumConflicts: 0
NumBranches: 48
WallTime: 0.0040030000000000005
